# VacationPy
---

## Import Libraries and Load the Weather and Coordinates Data

In [28]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

# Turn off warning 
# import warnings
# warnings.filterwarnings("ignore")

In [29]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,olonkinbyen,70.9221,-8.7187,0.08,86,56,6.57,SJ,1683852474
1,1,ingeniero jacobacci,-41.3427,-69.5448,9.62,68,90,4.27,AR,1683852475
2,2,luderitz,-26.6481,15.1594,15.14,78,100,10.80,NaN,1683852476
3,3,stanley,54.8680,-1.6985,9.84,97,75,3.60,GB,1683852476
4,4,salinopolis,-0.6136,-47.3561,25.78,85,35,2.39,BR,1683852194


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [52]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
         "Lng",
         "Lat",
         geo = True, 
         tiles = "ESRI",
         frame_width = 700, 
         frame_height =500, 
         size = "Humidity", 
         color = "City"
    
     )

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [39]:
# Narrow down cities that fit criteria and drop any results with null values
prefered_weather_df = city_data_df[ (city_data_df["Max Temp"] < 70) & (city_data_df["Max Temp"] > 21)]
# Drop any rows with null values
prefered_weather_df = prefered_weather_df.dropna()


# Display sample data
prefered_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,salinopolis,-0.6136,-47.3561,25.78,85,35,2.39,BR,1683852194
7,7,bandarbeyla,9.4942,50.8122,27.06,86,82,5.50,SO,1683852479
8,8,adamstown,-25.0660,-130.1015,21.93,59,100,7.77,PN,1683852481
10,10,puerto baquerizo moreno,-0.9000,-89.6000,26.22,87,100,3.93,EC,1683852482
11,11,bungu,-7.6337,39.0582,21.45,98,27,1.21,TZ,1683852483
...,...,...,...,...,...,...,...,...,...,...
572,572,soio,-6.1349,12.3689,26.33,91,34,2.40,AO,1683852683
573,573,bindki,26.0333,80.6000,28.91,13,0,2.39,IN,1683852684
574,574,palikir - national government center,6.9248,158.1611,32.41,70,40,3.09,FM,1683852684
575,575,port macquarie,-31.4333,152.9167,21.61,62,0,2.12,AU,1683852684


### Step 3: Create a new DataFrame called `hotel_df`.

In [47]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = prefered_weather_df[["City","Country", "Lat","Lng", "Humidity"]].copy() 

# Add an empty column, "Hotel Name," to the DataFrame to store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data


hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,salinopolis,BR,-0.6136,-47.3561,85,
7,bandarbeyla,SO,9.4942,50.8122,86,
8,adamstown,PN,-25.0660,-130.1015,59,
10,puerto baquerizo moreno,EC,-0.9000,-89.6000,87,
11,bungu,TZ,-7.6337,39.0582,98,
...,...,...,...,...,...,...
572,soio,AO,-6.1349,12.3689,91,
573,bindki,IN,26.0333,80.6000,13,
574,palikir - national government center,FM,6.9248,158.1611,70,
575,port macquarie,AU,-31.4333,152.9167,62,



### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [48]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row[ 'Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    #print(name_address)
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df
    

Starting hotel search
salinopolis - nearest hotel: Hotel Salinópolis
bandarbeyla - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
puerto baquerizo moreno - nearest hotel: Dory's House
bungu - nearest hotel: No hotel found
buurhakaba - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
bonthe - nearest hotel: No hotel found
zouerate - nearest hotel: فندق تيرس زمور
traverse city - nearest hotel: The Whiting
hadibu - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
hithadhoo - nearest hotel: Pebbles Inn
kailua-kona - nearest hotel: Kona Seaside Hotel
tolanaro - nearest hotel: Hôtel Mahavokey
la ronge - nearest hotel: La Ronge Hotel and Suites
bemidji - nearest hotel: Hampton Inn and Suites
greenland - nearest hotel: Quality Inn
fort mcmurray - nearest hotel: Nomad Hotel and Suites
rongelap - nearest hotel: No hotel found
utrik - nearest hotel: No 

parras de la fuente - nearest hotel: Posada Santa isabel
najaf - nearest hotel: مديرية مجاري النجف
north madison - nearest hotel: Clifty Inn
ugoofaaru - nearest hotel: No hotel found
mogadishu - nearest hotel: Villa Rays Restaurant & Hotel
howrah - nearest hotel: Sun India Guest House
batang - nearest hotel: Palm Beach Hotel
portel - nearest hotel: No hotel found
port-gentil - nearest hotel: Tara-Mé
brisas de zicatela - nearest hotel: Casa de Olas
lagunillas - nearest hotel: Palacio Suite
umm ruwaba - nearest hotel: No hotel found
levelland - nearest hotel: Best Inn Texas
malabar - nearest hotel: The Coogee Bay Hotel
itoman - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
pindiga - nearest hotel: No hotel found
wonosobo - nearest hotel: Hotel Arjuna
george town - nearest hotel: Page 63 hostel
acapulco de juarez - nearest hotel: Hotel del Valle
al wajh - nearest hotel: فندق رالين 2
thinadhoo - nearest hotel: No hotel found
olinda - nearest hotel: F

,City,Country,Lat,Lng,Humidity,Hotel Name
4,salinopolis,BR,-0.6136,-47.3561,85,Hotel Salinópolis
7,bandarbeyla,SO,9.4942,50.8122,86,No hotel found
8,adamstown,PN,-25.0660,-130.1015,59,No hotel found
10,puerto baquerizo moreno,EC,-0.9000,-89.6000,87,Dory's House
11,bungu,TZ,-7.6337,39.0582,98,No hotel found
...,...,...,...,...,...,...
572,soio,AO,-6.1349,12.3689,91,Hotel Maria Teres
573,bindki,IN,26.0333,80.6000,13,No hotel found
574,palikir - national government center,FM,6.9248,158.1611,70,Mangrove Bay Hotel
575,port macquarie,AU,-31.4333,152.9167,62,The Observatory


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [53]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
         "Lng",
         "Lat",
         geo = True, 
         tiles = "ESRI",
         frame_width = 500, 
         frame_height =500, 
         size = "Humidity", 
         color = "City",
         hover_cols= ['Country', 'Hotel Name']
    
     )

# Display the map
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)